# Agenda

1. What is Polars? What is its relationship with Pandas and the rest of the data ecosystem in Python?
2. Series
3. Data frames
4. Reading in CSV files
5. dtypes
6. Expressions
7. Selecting rows with `df.select`
8. Selecting columns
9. `df.with_columns`
10. `df.filter`
11. Sorting
12. Grouping
13. Optimizing queries and "lazy frames"

# What is Polars?

Data frames are everywhere, but they aren't native to very many languages. We're seeing a growing number of libraries that provide data frames outside of the core of languages. The bad news is that you have to install / choose which data-frame package you want to use in Python. The good news is that we have a few great options to choose from.

Pandas is the 900-pound gorilla in this space.  Polars is a relatively new entry, and it's written for (a) speed of execution, (b) builtin multithreading, (c) lazy loading of data, and (d) a very minimalist, elegant API.

The URL is pola.rs, because it's written in the Rust language.  The Python API that Polars exposes is very similar to the Pandas API, making it fairly easy for someone to move from Pandas to Polars.

I personally see Polars as great for when you have tons of data and/or execution speed is critical. If you're a snob who wants a very elegant API, then it's great, too. 



# Installing Polars

You can install it from PyPI using `pip`:

    pip install -U polars

You'll probably want to install the complete Polars package and dependencies and extras, using

    pip install -U 'polars[all]'

If you're using a Mac with Apple Silicon, Polars might die on you because your copy of Python was compiled for Intel, and is in compatibility mode.  If you're like me, and can't/won't/don't know how to recompile Python for Apple Silicon, you can just install a different Polars package from PyPI that takes this into account:

    pip install -U 'polars-lts-cpu[all]'
   

In [1]:
import polars as pl    # just as we use Pandas as pd, we use Polars as pl

# Series

A series is a 1D data structure, just like in Pandas. (There, behind the scenes, we have a NumPy array. That is *not* true in Polars!) 

We can create a Polars series just by invoking `pl.Series` on a list of values. Polars will (like Pandas) figure out what kinds of values we have, and set the dtype.

In [8]:
s = pl.Series(
    values=[10, 20, 30, 40, 50],
    name='numbers'
)

In [9]:
s

numbers
i64
10
20
30
40
50


# Some differences

1. No index! There is no index in Polars, period.
2. No name, although we can give it a name by passing the `name` argument, which comes *before* the data, unless you want both `name` and `values` to be keyword arguments, in which case they can be in any order.
3. The shape is always displayed at the top of the series (or data frame)
4. The dtype is displayed at the top of the series. And here, we see that the dtype is `i64`, aka `int64`

In [10]:
s = pl.Series(
    values=[10.5, 20.5, 30.5, 40.5, 50.5],
    name='numbers'
)

s

numbers
f64
10.5
20.5
30.5
40.5
50.5


In [11]:
# let's try another dtype

s = pl.Series(
    values=[10.5, 20.5, 30.5, 40.5, 50.5],
    name='numbers',
    dtype=pl.Float32   # use Polars dtypes, not NumPy dtypes and not strings
)

s

numbers
f32
10.5
20.5
30.5
40.5
50.5


In [18]:
# What if I have a mixture of types?

s = pl.Series(
    values=[10, 20.5, 30, 40.5, 50],
    dtype=pl.Int64
)

s

TypeError: unexpected value while building Series of type Int64; found value of type Float64: 20.5

Hint: Try setting `strict=False` to allow passing data with mixed types.

In [19]:
s = pl.Series(
    values=[10, 20.5, 30, 40.5, 50],
    dtype=pl.Int64,
    strict=False
)

s

""
i64
10
20
30
40
50


In [21]:
s = pl.Series(
    'hello out there from polars world!'.split()
)

s

""
str
"""hello"""
"""out"""
"""there"""
"""from"""
"""polars"""
"""world!"""


In [22]:
s.dtype

String

In [26]:
s1 = pl.Series(
    values=[10, 20, 30, 40, 50]
)

s
    

""
i64
10
20
30
40
50


In [27]:
s2 = pl.Series(
    values=[100, 200, 300, 400, 500]
)

s2
    

""
i64
100
200
300
400
500


In [29]:
# if we perform an operation on two series, assuming that they are both the same 
# length (i.e., shape), then the operations will be performed across the same
# indexes (yes, even though we don't have an official "index")

s1 + s2

""
i64
110
220
330
440
550


In [30]:
s1 + s2.head(2)

InvalidOperationError: cannot do arithmetic operation on series of different lengths: got 5 and 2

In [31]:
# can we do broadcast operations, arithmetic operations with a series and a scalar value?

s1 + 5

""
i64
15
25
35
45
55


In [32]:
s1 + 5.5

""
f64
15.5
25.5
35.5
45.5
55.5


In [33]:
s % 2 == 0

""
bool
true
true
true
true
true


In [34]:
s[2:4]

""
i64
30
40


# Methods we can run on our series

Just as Pandas provides a lot of methods for analyzing data on a series, Polars provides similar (or identical) methods.

In [36]:
s.mean()

30.0

In [37]:
s.std()

15.811388300841896

In [38]:
s.min()

10

In [39]:
s.max()

50

In [40]:
s.median()

30.0

In [41]:
s.quantile(0.25)

20.0

In [42]:
s.quantile(0.75)

40.0

In [43]:
s.count()

5

In [44]:
s.describe()

statistic,value
str,f64
"""count""",5.0
"""null_count""",0.0
"""mean""",30.0
"""std""",15.811388
"""min""",10.0
"""25%""",20.0
"""50%""",30.0
"""75%""",40.0
"""max""",50.0


In [45]:
s = pl.Series('this is a bunch of words for my Polars class'.split())

In [46]:
s.describe()

statistic,value
str,str
"""count""","""10"""
"""null_count""","""0"""
"""min""","""Polars"""
"""max""","""words"""


# Exercise: Polar series

1. Create a series containing the forecast high temperatures for where you live over the next 10 days.
2. What dtype does the series have? Force it to be ints. Force it to be floats.
3. Get the descriptive statistics for these values.
4. Calculate by how much each day's forecast high temp will differ from the mean and the median. 

In [47]:
high_temps = pl.Series(
    [36, 35, 34, 32, 32, 32, 32, 30, 30, 30]
)



In [48]:
high_temps

""
i64
36
35
34
32
32
32
32
30
30


In [51]:
high_temps = pl.Series(
    values=[36, 35, 34, 32, 32, 32, 32, 30, 30, 30],
    dtype=pl.Int8
)

high_temps

""
i8
36
35
34
32
32
32
32
30
30


In [52]:
high_temps = pl.Series(
    values=[36, 35, 34, 32, 32, 32, 32, 30, 30, 30],
    dtype=pl.Float32
)

high_temps

""
f32
36.0
35.0
34.0
32.0
32.0
32.0
32.0
30.0
30.0


In [53]:
high_temps.describe()

statistic,value
str,f64
"""count""",10.0
"""null_count""",0.0
"""mean""",32.299999
"""std""",2.110819
"""min""",30.0
"""25%""",30.0
"""50%""",32.0
"""75%""",34.0
"""max""",36.0


In [54]:
# broadcast operation
high_temps - high_temps.mean()

""
f32
3.700001
2.700001
1.700001
-0.299999
-0.299999
-0.299999
-0.299999
-2.299999
-2.299999


In [55]:
# broadcast operation
high_temps - high_temps.median()

""
f32
4.0
3.0
2.0
0.0
0.0
0.0
0.0
-2.0
-2.0


In [56]:
high_temps.describe()

statistic,value
str,f64
"""count""",10.0
"""null_count""",0.0
"""mean""",32.299999
"""std""",2.110819
"""min""",30.0
"""25%""",30.0
"""50%""",32.0
"""75%""",34.0
"""max""",36.0


# Data frames

A data frame is a 2D collection of data

- Once again, no index
- As in Pandas, every column is a series
- Every column needs to have a unique name

We can create a data frame in a few ways, including passing a dict whose keys are strings (the column names) and whose values are lists (or Polars series). Every list/series must contain the same number of values.

In [57]:
df = pl.DataFrame(
    {'high_temps': [36, 35, 34, 32, 32, 32, 32, 30, 30, 30],
     'low_temps': [24, 24, 23, 24, 24, 25, 23, 21, 21, 22]}
)

In [58]:
df

high_temps,low_temps
i64,i64
36,24
35,24
34,23
32,24
32,24
32,25
32,23
30,21
30,21


In [59]:
df.dtypes   # what are the dtypes of our columns?

[Int64, Int64]

In [60]:
df.columns

['high_temps', 'low_temps']

In [61]:
# I can retrieve a column using [] and the column name
df['high_temps']

high_temps
i64
36
35
34
32
32
32
32
30
30


In [62]:
# I can pass a list of columns, and get all of them back

df[['high_temps', 'low_temps']]

high_temps,low_temps
i64,i64
36,24
35,24
34,23
32,24
32,24
32,25
32,23
30,21
30,21


In [73]:
import numpy as np
np.random.seed(0)

df = pl.DataFrame(
    {'high_temp': [36, 35, 34, 32, 32, 32, 32, 30, 30, 30],
     'low_temp': [24, 24, 23, 24, 24, 25, 23, 21, 21, 22], 
     'random': np.random.randint(0, 1000, 10)}
)

In [74]:
df

high_temp,low_temp,random
i64,i64,i64
36,24,684
35,24,559
34,23,629
32,24,192
32,24,835
32,25,763
32,23,707
30,21,359
30,21,9


In [75]:
df[3]

high_temp,low_temp,random
i64,i64,i64
32,24,192


In [76]:
df[2:7]

high_temp,low_temp,random
i64,i64,i64
34,23,629
32,24,192
32,24,835
32,25,763
32,23,707


# How can we retrieve data?

We'll need *expressions* to do this, which are the Polars equivalent to boolean indexing in Pandas.

In [81]:
# let's find all rows where the high_temp is == 32
# Polars encourages us to say:
# - I want to select from my data frame
# - wherever the high_temp column is == 32

df.filter(    # this puts us into the context of df
    pl.col('high_temp') == 32
)

high_temp,low_temp,random
i64,i64,i64
32,24,192
32,24,835
32,25,763
32,23,707


In [82]:
df.filter(    # this puts us into the context of df
    pl.col('high_temp') > 32
)

high_temp,low_temp,random
i64,i64,i64
36,24,684
35,24,559
34,23,629


In [83]:
df.filter(    # this puts us into the context of df
    pl.col('high_temp') != 32
)

high_temp,low_temp,random
i64,i64,i64
36,24,684
35,24,559
34,23,629
30,21,359
30,21,9
30,22,723


In [84]:
df.filter(    # this puts us into the context of df
    pl.col('high_temp') > pl.col('high_temp').mean()
)

high_temp,low_temp,random
i64,i64,i64
36,24,684
35,24,559
34,23,629


In [88]:
# what if I want more than one condition?

# for example, I want days in which the high is >= mean, but the low is <= mean

df.filter(    
    (pl.col('high_temp') >= pl.col('high_temp').mean()) &
    (pl.col('low_temp') <= pl.col('low_temp').mean())
)

high_temp,low_temp,random
i64,i64,i64
34,23,629


In [90]:
df[5:0]

high_temp,low_temp,random
i64,i64,i64


In [91]:
text = 'hello out there'

text[5:0]

''

# Exercise: Polars data frames

1. Create a data frame with people in your family. There should be three columns:
    - name, with the person's name (as a string)
    - age, with the person's age in years (as an int)
    - shoe_size, with the person's shoe size (as an int)
2. Find all of the rows with people whose age is above the mean
3. Find all of the rows with people whose age is above the mean but whose shoe_size is below the mean

In [92]:
df = pl.DataFrame(
    {'name':['Reuven', 'Atara', 'Shikma', 'Amotz'],
     'age': [ 54,  23, 21, 18],
     'shoe_size': [46, 40, 40, 44]}
)

In [93]:
df

name,age,shoe_size
str,i64,i64
"""Reuven""",54,46
"""Atara""",23,40
"""Shikma""",21,40
"""Amotz""",18,44


In [94]:
df.filter(
    pl.col('age') > pl.col('age').mean()
)

name,age,shoe_size
str,i64,i64
"""Reuven""",54,46


In [95]:
df.filter(
    (pl.col('age') > pl.col('age').mean()) &
    (pl.col('shoe_size') < pl.col('shoe_size').mean())
)

name,age,shoe_size
str,i64,i64


In [96]:
# who is below-average age, but above-average shoe size?

df.filter(
    (pl.col('age') < pl.col('age').mean()) &
    (pl.col('shoe_size') > pl.col('shoe_size').mean())
)

name,age,shoe_size
str,i64,i64
"""Amotz""",18,44


# What if we want to cut down the columns?

The `filter` method is used to reduce the rows.

The `select` method is used to select the columns.

We can connect them via method chaining, and get only the rows and columns we want.

In [104]:
# Get the name and age
# of people whose shoe size is below average

df.filter(
    (pl.col('shoe_size') < pl.col('shoe_size').mean())    # keep only the rows that we mention here
).select(
    pl.col('name'),                                       # keep only the columns we mention here
    pl.col('age').alias('age_in_years'),
    (pl.col('age') * 12).alias('age_in_months')           # sneaky way to add a new column
)

name,age_in_years,age_in_months
str,i64,i64
"""Atara""",23,276
"""Shikma""",21,252


# Reading from a CSV file

It's very, very similar to what we do in Pandas: We use the `pl.read_csv` function.  It returns a data frame, in which every column of the CSV is a column in our data frame. There are many similar options, too -- to handle a lack of column names, rows we want to ignore, etc.

In [106]:
df.write_csv('family.csv')

In [107]:
!cat family.csv

name,age,shoe_size
Reuven,54,46
Atara,23,40
Shikma,21,40
Amotz,18,44


In [108]:
family_data = pl.read_csv('family.csv')

In [109]:
family_data

name,age,shoe_size
str,i64,i64
"""Reuven""",54,46
"""Atara""",23,40
"""Shikma""",21,40
"""Amotz""",18,44


In [110]:
(
    pl
    .read_csv('family.csv')
    .filter(
        (pl.col('shoe_size') < pl.col('shoe_size').mean())    # keep only the rows that we mention here
    ).select(
        pl.col('name'),                                       # keep only the columns we mention here
        pl.col('age').alias('age_in_years'),
        (pl.col('age') * 12).alias('age_in_months')           # sneaky way to add a new column
    )
)

name,age_in_years,age_in_months
str,i64,i64
"""Atara""",23,276
"""Shikma""",21,252


In [111]:
# let's load my favorite data file, a 10,000-line collection of NYC taxi information 

filename = 'taxi.csv'

df = pl.read_csv(filename)

ComputeError: could not parse `0.5` as dtype `i64` at column 'extra' (column number 14)

The current offset in the file is 653749 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `dtypes` argument
- setting `ignore_errors` to `True`,
- adding `0.5` to the `null_values` list.

Original error: ```remaining bytes non-empty```

In [112]:
df = pl.read_csv(filename, infer_schema_length=10000)

In [113]:
df.dtypes

[Int64,
 String,
 String,
 Int64,
 Float64,
 Float64,
 Float64,
 Int64,
 String,
 Float64,
 Float64,
 Int64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64,
 Float64]

In [114]:
df['extra']

extra
f64
0.0
0.0
0.0
0.0
0.0
…
0.5
0.5
0.5


In [115]:
df

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
i64,str,str,i64,f64,f64,f64,i64,str,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64
2,"""2015-06-02 11:19:29""","""2015-06-02 11:47:52""",1,1.63,-73.95443,40.764141,1,"""N""",-73.974754,40.754093,2,17.0,0.0,0.5,0.0,0.0,0.3,17.8
2,"""2015-06-02 11:19:30""","""2015-06-02 11:27:56""",1,0.46,-73.971443,40.758942,1,"""N""",-73.978539,40.761909,1,6.5,0.0,0.5,1.0,0.0,0.3,8.3
2,"""2015-06-02 11:19:31""","""2015-06-02 11:30:30""",1,0.87,-73.978111,40.738434,1,"""N""",-73.990273,40.745438,1,8.0,0.0,0.5,2.2,0.0,0.3,11.0
2,"""2015-06-02 11:19:31""","""2015-06-02 11:39:02""",1,2.13,-73.945892,40.773529,1,"""N""",-73.971527,40.76033,1,13.5,0.0,0.5,2.86,0.0,0.3,17.16
1,"""2015-06-02 11:19:32""","""2015-06-02 11:32:49""",1,1.4,-73.979088,40.776772,1,"""N""",-73.982162,40.758999,2,9.5,0.0,0.5,0.0,0.0,0.3,10.3
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,"""2015-06-01 00:12:59""","""2015-06-01 00:24:18""",1,2.7,-73.947792,40.814972,1,"""N""",-73.973358,40.783638,2,11.0,0.5,0.5,0.0,0.0,0.3,12.3
1,"""2015-06-01 00:12:59""","""2015-06-01 00:28:16""",1,4.5,-74.004066,40.747818,1,"""N""",-73.953758,40.779285,1,16.0,0.5,0.5,3.0,0.0,0.3,20.3
2,"""2015-06-01 00:13:00""","""2015-06-01 00:37:25""",1,5.59,-73.994377,40.766102,1,"""N""",-73.903206,40.750546,2,21.0,0.5,0.5,0.0,0.0,0.3,22.3


# Exercise: Taxi CSV stuff

1. Load `taxi.csv` (you can get it from GitHub, or from https://files.lerner.co.il/data-science-exercise-files.zip) into a data frame.
2. Find the mean `trip_distance` where the `passenger_count` is > 1.
3. How many trips had a `total_amount` of <= 0?

In [116]:
filename = 'taxi.csv'

df = pl.read_csv(filename, infer_schema_length=10000)

In [120]:
# find the mean trip_distance where passenger_count is > 1

(
    df
    .filter(    # keep only those rows where passenger_count > 1
        pl.col('passenger_count') > 1
    ).select(   # keep only the trip_distance column
        pl.col('trip_distance')
    )
    .mean()     # invoke a method on that column
)

trip_distance
f64
3.328412


In [121]:
# what if I wanted trip_distance and total_amount?

# find the mean trip_distance where passenger_count is > 1

(
    df
    .filter(    # keep only those rows where passenger_count > 1
        pl.col('passenger_count') > 1
    ).select(   
        pl.col('trip_distance'),
        pl.col('total_amount')
    )
    .mean()     # invoke a method on those two columns
)

trip_distance,total_amount
f64,f64
3.328412,18.021774


In [125]:
# How many trips had a total_amount of <= 0?

(
    df
    .filter(  # keep only those rows where total_amount <= 0
        pl.col('total_amount') <= 0
    )
    .select(
        'VendorID'   # only keep this column, for easier reading of the results
    )
    .count()
)

VendorID
u32
3


# Next up

1. `with_columns`
2. Sorting
3. Grouping
4. Pivot tables
5. Lazy/eager API

Resume at :35

In [126]:
df.head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
i64,str,str,i64,f64,f64,f64,i64,str,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64
2,"""2015-06-02 11:19:29""","""2015-06-02 11:47:52""",1,1.63,-73.95443,40.764141,1,"""N""",-73.974754,40.754093,2,17.0,0.0,0.5,0.0,0.0,0.3,17.8
2,"""2015-06-02 11:19:30""","""2015-06-02 11:27:56""",1,0.46,-73.971443,40.758942,1,"""N""",-73.978539,40.761909,1,6.5,0.0,0.5,1.0,0.0,0.3,8.3
2,"""2015-06-02 11:19:31""","""2015-06-02 11:30:30""",1,0.87,-73.978111,40.738434,1,"""N""",-73.990273,40.745438,1,8.0,0.0,0.5,2.2,0.0,0.3,11.0
2,"""2015-06-02 11:19:31""","""2015-06-02 11:39:02""",1,2.13,-73.945892,40.773529,1,"""N""",-73.971527,40.76033,1,13.5,0.0,0.5,2.86,0.0,0.3,17.16
1,"""2015-06-02 11:19:32""","""2015-06-02 11:32:49""",1,1.4,-73.979088,40.776772,1,"""N""",-73.982162,40.758999,2,9.5,0.0,0.5,0.0,0.0,0.3,10.3


In [128]:
(
    df
    .with_columns(
        (pl.col('passenger_count') * 2).alias('pc_times_2')
    )
)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pc_times_2
i64,str,str,i64,f64,f64,f64,i64,str,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,i64
2,"""2015-06-02 11:19:29""","""2015-06-02 11:47:52""",1,1.63,-73.95443,40.764141,1,"""N""",-73.974754,40.754093,2,17.0,0.0,0.5,0.0,0.0,0.3,17.8,2
2,"""2015-06-02 11:19:30""","""2015-06-02 11:27:56""",1,0.46,-73.971443,40.758942,1,"""N""",-73.978539,40.761909,1,6.5,0.0,0.5,1.0,0.0,0.3,8.3,2
2,"""2015-06-02 11:19:31""","""2015-06-02 11:30:30""",1,0.87,-73.978111,40.738434,1,"""N""",-73.990273,40.745438,1,8.0,0.0,0.5,2.2,0.0,0.3,11.0,2
2,"""2015-06-02 11:19:31""","""2015-06-02 11:39:02""",1,2.13,-73.945892,40.773529,1,"""N""",-73.971527,40.76033,1,13.5,0.0,0.5,2.86,0.0,0.3,17.16,2
1,"""2015-06-02 11:19:32""","""2015-06-02 11:32:49""",1,1.4,-73.979088,40.776772,1,"""N""",-73.982162,40.758999,2,9.5,0.0,0.5,0.0,0.0,0.3,10.3,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,"""2015-06-01 00:12:59""","""2015-06-01 00:24:18""",1,2.7,-73.947792,40.814972,1,"""N""",-73.973358,40.783638,2,11.0,0.5,0.5,0.0,0.0,0.3,12.3,2
1,"""2015-06-01 00:12:59""","""2015-06-01 00:28:16""",1,4.5,-74.004066,40.747818,1,"""N""",-73.953758,40.779285,1,16.0,0.5,0.5,3.0,0.0,0.3,20.3,2
2,"""2015-06-01 00:13:00""","""2015-06-01 00:37:25""",1,5.59,-73.994377,40.766102,1,"""N""",-73.903206,40.750546,2,21.0,0.5,0.5,0.0,0.0,0.3,22.3,2


In [130]:
# more natural (I think) to use **kwargs

(
    df
    .with_columns(
        pc_times_2 = pl.col('passenger_count') * 2,
        dollars_per_passenger = pl.col('total_amount') / pl.col('passenger_count')
    )
)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pc_times_2,dollars_per_passenger
i64,str,str,i64,f64,f64,f64,i64,str,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,i64,f64
2,"""2015-06-02 11:19:29""","""2015-06-02 11:47:52""",1,1.63,-73.95443,40.764141,1,"""N""",-73.974754,40.754093,2,17.0,0.0,0.5,0.0,0.0,0.3,17.8,2,17.8
2,"""2015-06-02 11:19:30""","""2015-06-02 11:27:56""",1,0.46,-73.971443,40.758942,1,"""N""",-73.978539,40.761909,1,6.5,0.0,0.5,1.0,0.0,0.3,8.3,2,8.3
2,"""2015-06-02 11:19:31""","""2015-06-02 11:30:30""",1,0.87,-73.978111,40.738434,1,"""N""",-73.990273,40.745438,1,8.0,0.0,0.5,2.2,0.0,0.3,11.0,2,11.0
2,"""2015-06-02 11:19:31""","""2015-06-02 11:39:02""",1,2.13,-73.945892,40.773529,1,"""N""",-73.971527,40.76033,1,13.5,0.0,0.5,2.86,0.0,0.3,17.16,2,17.16
1,"""2015-06-02 11:19:32""","""2015-06-02 11:32:49""",1,1.4,-73.979088,40.776772,1,"""N""",-73.982162,40.758999,2,9.5,0.0,0.5,0.0,0.0,0.3,10.3,2,10.3
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,"""2015-06-01 00:12:59""","""2015-06-01 00:24:18""",1,2.7,-73.947792,40.814972,1,"""N""",-73.973358,40.783638,2,11.0,0.5,0.5,0.0,0.0,0.3,12.3,2,12.3
1,"""2015-06-01 00:12:59""","""2015-06-01 00:28:16""",1,4.5,-74.004066,40.747818,1,"""N""",-73.953758,40.779285,1,16.0,0.5,0.5,3.0,0.0,0.3,20.3,2,20.3
2,"""2015-06-01 00:13:00""","""2015-06-01 00:37:25""",1,5.59,-73.994377,40.766102,1,"""N""",-73.903206,40.750546,2,21.0,0.5,0.5,0.0,0.0,0.3,22.3,2,22.3


# Filtering, seletijg